<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/database.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Integrating pandas with SingleStoreDB</h1>
    </div>
</div>

This notebook will show how to move data from a pandas `DataFrame` into SingleStoreDB as well
as downloading a SingleStoreDB query to a pandas `DataFrame`. It should be noted that this
is only intended for relatively small data sets and to do processing that can't otherwise
be done in SingleStoreDB itself. Moving data to the client for processing should only be done
when there is no other alternative in the database.

In [ ]:
import ibis
import pandas as pd
import singlestoredb as s2
import sqlalchemy as sa

## Create a database

We need to create a database to work with in the following examples.

In [ ]:
%%sql
DROP DATABASE IF EXISTS pandas_integration;

CREATE DATABASE IF NOT EXISTS pandas_integration;

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Make sure to select the <tt>pandas_integration</tt> database from the drop-down menu at the top of this notebook.
        It updates the <tt>connection_url</tt> to connect to that database.</p>
    </div>
</div>

## Database connections

In the notebooks environment, the connection string for the currently selected database is kept
in the `connection_url` variable as well as the `SINGLESTOREDB_URL` environment variable.
The connection variables are accessed automatically within the SingleStoreDB Python packages
so that you do not need connection parameters when connecting.

In the following sections, we will connect to SingleStoreDB using each of the packages and demonstrate
techniques for moving data between pandas and SingleStoreDB.

## The Iris data set

We'll be using the Iris data set for the following examples. This data set includes five columns: `sepal_length`,
`sepal_width`, `petal_length`, `petal_width` and `class`.

In [ ]:
iris = pd.read_csv('https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/'
                   'master/notebooks/integrating-with-pandas/data/iris.csv')
iris

As you can see below, the first four columns are floats and the last column is a string 
(represented as an `object` in `DataFrame`s). The `petal_width` column has 5 missing values.

In [ ]:
iris.info()

## Moving data between SingleStoreDB and pandas `DataFrame`s

Moving data from pandas `DataFrame`s to SingleStoreDB tables can be done in various ways from Python and even 
from each of the packages described here. This reference is to show the best techniques when using each 
package.

It should be noted that moving data back-and-forth between pandas and SingleStoreDB should only be done when
absolutely needed since this can be a major bottleneck when working with and analyzing data. The hope is that
the features of SingleStoreDB are sufficient enough to alleviate the need to do much processing (if any) on 
the client machine.

### SingleStoreDB Python

The core library is the SingleStoreDB Python package. This is the package that all other SingleStoreDB
packages are built on. To connect, simply call the `connect` function.

In [ ]:
s2_conn = s2.connect()

Since the core library is a fairly low-level interface to SingleStoreDB, most operations are done simply by sending
SQL code.

#### Creating a table

Because we are using a low-level driver, creating a table is just done using SQL code. We'll use the information
about the data set above to construct a `CREATE TABLE` statement.

In [ ]:
s2_cur = s2_conn.cursor()
s2_cur.execute(r'''
    CREATE TABLE IF NOT EXISTS iris (
        sepal_length DOUBLE,
        sepal_width DOUBLE,
        petal_length DOUBLE,
        petal_width DOUBLE,
        class TEXT
    )
''')

#### Upload the data from a `DataFrame`

Now that we have a table, we can populate it with data from the `DataFrame`. Again, we will use
SQL statements to do this. The Python client can execute single SQL statements using the 
`execute` method as used above, but since we are uploading multiple rows of data it is better
to use the `executemany` method since it is optimized for this purpose. 

In [ ]:
# Construct the list of column names
cols = ', '.join(iris.columns)

# Construct a list of value placeholders for the INSERT statement
values = ', '.join(['%s'] * len(iris.columns))

# Get data as a list of tuples (not including the index)
data = list(iris.itertuples(index=False))

# Execute the INSERT statement
s2_cur.executemany(f'INSERT INTO iris({cols}) VALUES ({values})', data)

We can select a sample of the rows to see that the data is now in SingleStoreDB.

In [ ]:
s2_cur.execute('SELECT * FROM iris LIMIT 10')
for row in s2_cur:
    print(row)

#### Downloading the data to a `DataFrame`

We can download the data to a pandas `DataFrame` simply by selecting all columns of data, 
fetching all of the rows, and passing them to the `DataFrame` constructor.

In [ ]:
s2_cur.execute('SELECT * FROM iris')

# Use the `description` attribute to get the column names
names = [x[0] for x in s2_cur.description]

s2_iris_df = pd.DataFrame(list(s2_cur), columns=names)
s2_iris_df

In [ ]:
s2_iris_df.info()

Now that we have demonstrated uploading and downloading data from a pandas `DataFrame` using the
SingleStoreDB Python client, we can drop the table and move on to SQLAlchemy.

In [ ]:
s2_cur.execute('DROP TABLE IF EXISTS iris')

### SQLAlchemy

In addition to the core Python library, you can use SQLAlchemy to connect to SingleStoreDB. Typically, when 
using SQLAlchemy, you would use the SQLAlchemy `create_engine` function to create an engine, then call `connect`
on the engine to create connections from a pool. The SingleStoreDB Python package also has a `create_engine`
function that does the same thing, however, it extends the default ability by allow you to use the 
`SINGLESTOREDB_URL` environment variable as the connection string so that no parameters are needed for
`create_engine` when used in the notebooks environment.

In [ ]:
sa_eng = s2.create_engine()
sa_conn = sa_eng.connect()

#### Uploading the data from a `DataFrame`

Uploading data from a `DataFrame` using SQLAlchemy is much easier than the lower-level Python library.
The pandas library itself has the ability to communicate with SingleStoreDB using a SQLAlchemy connection.
In this case, the `DataFrame` can create the table and populate it in one step using the `to_sql` method.
The `to_sql` method has various options to modify its behavior [documented on the pandas web site](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html).

In [ ]:
iris.to_sql('iris', con=sa_conn, index=False, if_exists='replace')

We can verify the data is in SingleStoreDB with a simple `SELECT` statement.

In [ ]:
for row in sa_conn.execute('SELECT * FROM iris LIMIT 10'):
    print(row)

It is also possible to use SQLAlchemy expressions to query the table rather than raw SQL strings.

In [ ]:
# Create a metadata object
meta = sa.MetaData(bind=sa_eng)
sa.MetaData.reflect(meta)

# Get the iris table from reflected data
sa_iris = meta.tables['iris']

# Query the iris table
query = sa.select(sa_iris).limit(10)

# Print results
for row in sa_conn.execute(query):
    print(row)

#### Downloading the data to a `DataFrame`

Downloading data to a pandas `DataFrame` is very simple. The result of the `execute` method can 
be passed directly to the pandas `DataFrame` constructor.

In [ ]:
# Reset query to not include the limit
query = sa.select(sa_iris)

sa_iris_df = pd.DataFrame(sa_conn.execute(query))
sa_iris_df

In [ ]:
sa_iris_df.info()

Now that we have demonstrated using SQLAlchemy to upload and download pandas `DataFrames` we can drop 
the table and move on to Ibis.

In [ ]:
sa_iris.drop()

### Ibis (SingleStoreDB DataFrame)

The Ibis package allows you to treat tables in SingleStoreDB as `DataFrames`. The `DataFrame` expressions
are used to build lazy expressions which generate SQL statements that get submitted to SingleStoreDB
only when you want to see the results of a query. Ibis using SQLAlchemy connections behind-the-scenes.

In [ ]:
ibis_conn = ibis.singlestoredb.connect()

#### Uploading the data from a `DataFrame`

Ibis is intended for tight integration with pandas, so it is no surprise that uploading a 
pandas `DataFrame` with Ibis is straight-forward. 

If you are not familiar with Ibis, you may notice the `execute` call at the end of this cell.
Ibis creates expressions in memory on the client machine until a view of the data is 
explicitly asked for. Once you explicitly ask for a query to be executed, it then generates
and submits the SQL code for the expression behind-the-scenes.

In this case, the `ibis_iris` object is a `DataFrame`-like object that is lazily constructing
the requested expression until `execute` is called on it. In the case of this example, uploading
and downloading 

In [ ]:
ibis_iris = ibis_conn.create_table('iris', iris, force=True)
ibis_iris.limit(10).execute()

One way to see the SQL that gets submitted during `execute` is to compile the expression
and print it. Ibis also has a options to display SQL queries as they are submitted.

In [ ]:
print(ibis_iris.compile())

The information about the table can be retrieved much like in a local pandas `DataFrame`.

In [ ]:
ibis_iris.info().execute()

#### Downloading the data from a `DataFrame`

The output from evaluating Ibis expressions returns a `DataFrame`, so we have already demonstrated
downloading data, but here is the code again.

In [ ]:
ibis_iris_df = ibis_iris.execute()
ibis_iris_df

We have demonstrated both uploading and downloading pandas `DataFrames` using Ibis, so 
we can drop the table now.

In [ ]:
ibis_conn.drop_table('iris')

## Conclusion

We have shown how to upload and download data from a pandas `DataFrame` to and from SingleStoreDB
using the SingleStoreDB Python client, SQLAlchemy, and Ibis. These techniques should enable you to
integrate your pandas workflows with SingleStoreDB.

In [ ]:
%%sql
DROP DATABASE IF EXISTS pandas_integration;

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
    <div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>
</div>